# importing data from database

In [60]:
import numpy as np
import pandas as pd
from SportsExperiencePlatform.data import connect_db, get_data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import save_npz, load_npz

In [61]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [69]:
conn = connect_db()

if conn:
    users_df, events_df, ahoy_events_df = get_data(conn)

/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlche

In [70]:
users_df.shape, events_df.shape, ahoy_events_df.shape

((11, 16), (850, 17), (24, 6))

In [79]:
users_df.columns

Index(['id', 'email', 'encrypted_password', 'reset_password_token',
       'reset_password_sent_at', 'remember_created_at', 'created_at',
       'updated_at', 'first_name', 'last_name', 'description', 'date_of_birth',
       'gender', 'location', 'latitude', 'longitude'],
      dtype='object')

In [81]:
user_loc_df = users_df[['location','latitude','longitude']]
user_loc_df.head(15)

,location,latitude,longitude
0,Munich,48.135124,11.581981
1,Berlin,52.520008,13.404954
2,Berlin,52.520008,13.404954
3,Hamburg,53.551086,9.993682
4,Hamburg,53.551086,9.993682
5,Cologne,50.937531,6.960279
6,Cologne,50.937531,6.960279
7,Stuttgart,48.775845,9.182932
8,None,NaN,NaN
9,Munich,48.135124,11.581981


In [90]:
users_df.at[8, 'location'] = 'Berlin'

In [95]:
users_df.at[8, 'latitude'] = 52.520008 
users_df.at[8, 'longitude'] = 13.404954
users_df.at[11, 'location'] = 'Berlin'
users_df.at[11, 'location'] = 'Berlin'
users_df.at[11, 'latitude'] = 52.520008 
users_df.at[11, 'longitude'] = 13.404954
users_df.at[10, 'location'] = 'Berlin'
users_df.at[10, 'latitude'] = 52.520008 
users_df.at[10, 'longitude'] = 13.404954

In [96]:
users_df.head(15)

,id,email,encrypted_password,reset_password_token,reset_password_sent_at,remember_created_at,created_at,updated_at,first_name,last_name,description,date_of_birth,gender,location,latitude,longitude
0,55.0,johanna@gmail.com,$2a$12$CqtujmUxnctn4PYh/LQ7Yen4N9zGBa8mAkW5lJk...,None,None,None,2022-05-28 12:26:38.267750,2022-05-28 12:26:38.283275,Johanna,Maier,"Hi, my name is Johanna! I want to go on my nex...",1993-01-25,female,Munich,48.135124,11.581981
1,56.0,anjali@gmail.com,$2a$12$zoaJySL5quI21u316oCrzuwLap5n3iaoJoFEt8O...,None,None,None,2022-05-28 12:26:39.191914,2022-05-28 12:26:39.203232,Anjali,Kumar,"Hi, my name is Anjali! I want to go on my next...",1993-01-25,female,Berlin,52.520008,13.404954
2,57.0,alexandra@gmail.com,$2a$12$EFJExS3e79KnS1Sj5SU7iuA0oE4Fe3cghnIcWCj...,None,None,None,2022-05-28 12:26:40.648549,2022-05-28 12:26:40.656142,Alexandra,Stroe,"Hi, my name is Alexandra! I want to go on my n...",1993-01-25,female,Berlin,52.520008,13.404954
3,58.0,ieva@gmail.com,$2a$12$2KfdqVXZIU7RbiktQW1GAuaBnIcIfHBSc68kwWc...,None,None,None,2022-05-28 12:26:41.476255,2022-05-28 12:26:41.484819,Ieva,Jirgensone,"Hi, my name is Ieva! I want to go on my next s...",1993-01-25,female,Hamburg,53.551086,9.993682
4,60.0,julian@gmail.com,$2a$12$JaT8SCjL4GJ4gokRU/GO5.1YpDCt00/GwNj8ChD...,None,None,None,2022-05-28 12:26:43.051327,2022-05-28 12:26:43.062398,Julian,Muhlbauer,"Hi, my name is Julian! I want to go on my next...",1993-01-25,male,Hamburg,53.551086,9.993682
5,61.0,shreetama@gmail.com,$2a$12$Y3BZQdV7MhHLkHWNjXNI2.0/xszGFofvoLFtvwq...,None,None,None,2022-05-28 12:26:43.898237,2022-05-28 12:26:43.910509,Shreetama,Karmakar,"Hi, my name is Shreetama! I want to go on my n...",1993-01-25,female,Cologne,50.937531,6.960279
6,62.0,andrew@gmail.com,$2a$12$WQNQrH8cu5Pmvk228dOzCONyjixPUEhV.snxt1j...,None,None,None,2022-05-28 12:26:44.867437,2022-05-28 12:26:44.878280,Andrew,Erlanger,"Hi, my name is Andrew! I want to go on my next...",1993-01-25,male,Cologne,50.937531,6.960279
7,63.0,killian@gmail.com,$2a$12$EGWPW0BWsJk07er5uGiXjeNaLFqubjkIvSaq6VS...,None,None,None,2022-05-28 12:26:45.774583,2022-05-28 12:26:45.784615,Killian,Dectot,"Hi, my name is Killian! I want to go on my nex...",1993-01-25,male,Stuttgart,48.775845,9.182932
8,64.0,shreetama1985@gmail.com,$2a$12$8O0AVmviqFX4G.eP/ual6uGUfv6wiq3C5Y.5VcS...,None,None,None,2022-05-28 13:37:08.002699,2022-05-28 13:37:08.002699,Shreetama,Karmakar,None,None,None,Berlin,52.520008,13.404954
9,59.0,daniel@gmail.com,$2a$12$/DCExarGzhiUF8AS44/Tt.5P4SLPySCFgkqStUW...,None,None,None,2022-05-28 12:26:42.260631,2022-05-28 12:26:42.268129,Daniel,Angerer,"Hi, my name is Daniel! I want to go on my next...",1993-01-25,male,Munich,48.135124,11.581981


In [97]:
ahoy_events_df.shape

(24, 6)

In [98]:
ahoy_events_df.user_id.unique()

array([64, 55, 59, 65])

In [99]:
ahoy_events_df.head()

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517


# simple tfidf model: using the raw description

## advanced cleaning of data

In [8]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from googletrans import Translator

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreetamakarmakar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:

# def to_english(string):
    

# # result.src
# if translator.detect(string).lang != 'en':
#     result = translator.translate(string)
#     result.text

In [11]:
from googletrans import Translator
def language_translation(string):
    
    translator = Translator()
    if translator.detect(string).lang != 'en':
        result = translator.translate(string)
        return result.text
    else:
        return string

In [12]:
language_translation('yo no sabe')

'I do not know'

In [13]:
ahoy_events_df.head()

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517


In [14]:
users_df.head().T

,0,1,2,3,4
id,55,56,57,58,59
email,johanna@gmail.com,anjali@gmail.com,alexandra@gmail.com,ieva@gmail.com,daniel@gmail.com
encrypted_password,$2a$12$CqtujmUxnctn4PYh/LQ7Yen4N9zGBa8mAkW5lJk...,$2a$12$zoaJySL5quI21u316oCrzuwLap5n3iaoJoFEt8O...,$2a$12$EFJExS3e79KnS1Sj5SU7iuA0oE4Fe3cghnIcWCj...,$2a$12$2KfdqVXZIU7RbiktQW1GAuaBnIcIfHBSc68kwWc...,$2a$12$/DCExarGzhiUF8AS44/Tt.5P4SLPySCFgkqStUW...
reset_password_token,None,None,None,None,None
reset_password_sent_at,None,None,None,None,None
remember_created_at,None,None,None,None,None
created_at,2022-05-28 12:26:38.267750,2022-05-28 12:26:39.191914,2022-05-28 12:26:40.648549,2022-05-28 12:26:41.476255,2022-05-28 12:26:42.260631
updated_at,2022-05-28 12:26:38.283275,2022-05-28 12:26:39.203232,2022-05-28 12:26:40.656142,2022-05-28 12:26:41.484819,2022-05-28 12:26:42.268129
first_name,Johanna,Anjali,Alexandra,Ieva,Daniel
last_name,Maier,Kumar,Stroe,Jirgensone,Angerer


In [15]:
events_df['translated_title'] = events_df['title'].apply(lambda x: language_translation(x))
events_df['translated_description'] = events_df['description'].apply(lambda x: language_translation(x))

In [16]:
8# load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text,stem="None"):

    final_string = ""

    # make lower
    text = text.lower()
    
    # remove http
    text = re.sub(r"http.*\.[a-z]{2,3}","",text)

    # remove www
    text = re.sub(r"www.*\.[a-z]{2,3}","",text)
    
    # remove line breaks
    text = re.sub(r'\n','',text)

    # remove puncuation
    translator = str.maketrans('','',string.punctuation)
    text = text.translate(translator)

    # remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    text_filtered = [word for word in text if not word in useless_words]

    # remove numbers
    text_filtered = [re.sub(r'\w*\d\w*','',w) for w in text_filtered]

    # stem or lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string
        
        

In [17]:
events_df['clean_description'] = events_df['translated_description'].apply(lambda x: clean_string(x))
events_df['clean_title'] = events_df['translated_title'].apply(lambda x: clean_string(x))

In [25]:
ahoy_events_df.user_id.unique()

array([64, 55, 59])

In [ ]:
users_

In [18]:
events_df.shape[0]

850

In [22]:
events_df['description'][190]

"On this hike you can join a small group of passionate campers and hikers from this event: [https://www.meetup.com/BerlinTravel-Meetup/events/286074116/](https://www.meetup.com/BerlinTravel-Meetup/events/286074116/). On our way, there will be several swimming opportunities - fingers crossed that the weather will be good enough!\n\n**Tour highlights**\n\n* Lake Großer Wentowsee\n* Many little artificial lakes\n* [Ziegeleipark Mildenberg](https://www.ziegeleipark.de/)\n* Complete route: [View in browser](https://t1p.de/czsch) & [download as GPX/KML](https://easyupload.io/m/ohtsck)\n\n**How to get there**\n\n* Take the RE5 with direction Neustrelitz, arrival at Dannenwalde (bei Gransee) will be at 11:07.\n* We'll meet on the platform.\n* I recommend to buy a 9€ ticket which will be valid for the entire month in all of Germany (except intercity trains like ICE, IC, ...). If you already have an Abo for Berlin AB/ABC/whatever, this will count as a valid ticket as well.\n\n**Please bring**\n\

In [26]:
events_df['clean_description'][190]

'hike join small group passionate campers hikers event  way several swimming opportunities fingers crossed weather good enoughtour highlights lake großer wentowsee many little artificial lakes ziegeleipark mildenberg complete route view browsermohtsckhow get take  direction neustrelitz arrival dannenwalde bei gransee  well meet platform recommend buy € ticket valid entire month germany except intercity trains like ice ic already abo berlin ababcwhatever count valid ticket wellplease bring something eat drink swimming equipment'

#  recommender combining both title and description

## mashup of title and description

In [27]:
import random
'''this may be a bit dangerous but doing just for fun!!!'''
def random_generate_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 5:
        names = names[:5]
            
        
    return names

def random_generate_long_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 200:
        names = names[:200]
            
        
    return names

In [28]:
events_df['random_title'] = events_df['clean_title'].apply(lambda x: random_generate_list(x))
events_df['random_description'] = events_df['clean_description'].apply(lambda x: random_generate_long_list(x))

In [31]:
(' '.join(events_df["random_title"][0]) + ' ' + ' '.join(events_df["random_description"][190])).split()

['guild',
 'wandsbek',
 'bowling',
 'except',
 'direction',
 'passionate',
 'lakes',
 'bring',
 'swimming',
 'ice',
 'good',
 'equipment',
 'like',
 'ticket',
 'entire',
 'browsermohtsckhow',
 'abo',
 'artificial',
 'ziegeleipark',
 'ticket',
 'join',
 'view',
 'intercity',
 'drink',
 'month',
 'valid',
 'count',
 'großer',
 'hike',
 'already',
 'dannenwalde',
 'ic',
 'platform',
 'small',
 'bei',
 'neustrelitz',
 'weather',
 'opportunities',
 'little',
 'get',
 'eat',
 'recommend',
 'complete',
 'arrival',
 'well',
 'trains',
 'group',
 'several',
 'mildenberg',
 'enoughtour',
 'wellplease',
 'ababcwhatever',
 'many',
 'buy',
 'hikers',
 'way',
 'campers',
 'swimming',
 'germany',
 'route',
 'gransee',
 '€',
 'valid',
 'wentowsee',
 'fingers',
 'meet',
 'berlin',
 'something',
 'lake',
 'crossed',
 'take',
 'event',
 'highlights']

In [32]:
import random
def create_soup(x):
    soup = (" ".join(x["random_title"]) + " " + " ".join(x["random_description"])).split()
    random.shuffle(soup)
    return ' '.join(soup)
# df["soup"] = df.apply(create_soup, axis = 1)
# df["soup"].head(3)

In [33]:
events_df['soup'] = events_df.apply(create_soup, axis = 1)

In [35]:
events_df.soup[190]

'direction bring valid swimming wentowsee view ziegeleipark valid eat meet way ice swimming drink wellplease hike small lakes swim platform ziegeleipark count hike artificial hikers little recommend mildenberg highlights ababcwhatever several group ticket neustrelitz browsermohtsckhow abo entire fingers € dannenwalde gransee take get opportunities intercity equipment good enoughtour berlin route event except mildenberg month like passionate something km arrival already crossed weather complete germany well ic ticket join lake buy trains many bei campers großer'

## tfidf on clean data

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf = TfidfVectorizer(stop_words="english")

In [37]:
tfidf_matrix = tfidf.fit_transform(events_df["soup"])
tfidf_matrix.shape

(850, 7583)

In [38]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [39]:
ahoy_events_df.head()

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517


In [40]:
title_to_index = pd.Series(events_df.index, index = events_df["title"]).drop_duplicates()

In [41]:
ahoy_events_df.user_id.unique()

array([64, 55, 59])

In [42]:
# event_idx = ahoy_events_df[ahoy_events_df['user_id']==46].iloc[0].properties['offer']
# # now we get the title of the event
# title = events_df[events_df['id']==event_idx].title

In [43]:
events_df.head()

,id,title,description,price_per_person,capacity,address,latitude,longitude,is_external,url,...,user_id,tag_id,external_image_url,translated_title,translated_description,clean_description,clean_title,random_title,random_description,soup
0,2336,Gilde Bowling Wandsbek,no description,0,1.0,no address provided,NaN,NaN,True,https://www.meetup.com/bowling-und-freizeit-gr...,...,NaN,124,https://secure-content.meetupstatic.com/images...,Guild Bowling Wandsbek,no description,description,guild bowling wandsbek,"[guild, wandsbek, bowling]",[description],bowling description wandsbek guild
1,2337,Gilde Bowling Wandsbek,no description,0,1.0,no address provided,NaN,NaN,True,https://www.meetup.com/bowling-und-freizeit-gr...,...,NaN,124,https://secure-content.meetupstatic.com/images...,Guild Bowling Wandsbek,no description,description,guild bowling wandsbek,"[guild, bowling, wandsbek]",[description],guild bowling description wandsbek
2,2338,Bowlingabend,Zwei Bahnen sind für uns ab 19 Uhr im Gilde-Bo...,0,NaN,Wandsbeker Zollstraße 25-29,53.575390,10.074991,True,https://www.meetup.com/bowling-gruppe/events/2...,...,NaN,124,https://secure-content.meetupstatic.com/images...,Bowlingabend,Two lanes are reserved for us from 7 p.m. in t...,two lanes reserved us pm gildebowling wandsbe...,bowlingabend,[bowlingabend],"[eat, gildebowling, together, something, sit, ...",us drink sit reserved something hours wandsbek...
3,2339,Return To Love Virtual Retreat,"""Jess X Goh guided me in a process that made m...",0,NaN,no address provided,-8.521147,179.196200,True,https://www.meetup.com/the-socializer-hamburg/...,...,NaN,124,https://secure-content.meetupstatic.com/images...,Return To Love Virtual Retreat,"""Jess X Goh guided me in a process that made m...",jess x goh guided process made unleash suppres...,return love virtual retreat,"[love, retreat, virtual, return]","[temple, invitation, contain, healing, one, no...",receive soul knit absolutely living successful...
4,2340,Gaelic Football Training Hamburg m/f/d - all w...,Beginners welcome to join Gaelic football trai...,0,3.0,no address provided,53.594521,10.017610,True,https://www.meetup.com/Irish-Sports-Hurling-Ca...,...,NaN,125,https://secure-content.meetupstatic.com/images...,Gaelic Football Training Hamburg m/f/d - all w...,Beginners welcome to join Gaelic football trai...,beginners welcome join gaelic football trainin...,gaelic football training hamburg mfd welcome,"[hamburg, mfd, welcome, football, training]","[take, handpassing, irish, field, derived, foo...",move hamburg gaelic stadtpark required rectang...


In [44]:
import datetime

In [112]:
from datetime import date

def get_title(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_idx = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    title = events_df[events_df['id']==event_idx].title
    
    return title

# def get_current_date():
    
    
    
# def get_user_loc():   

def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    '''This recommender finds out 20 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    title = get_title(user_id, df = ahoy_events_df)
    
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    
    return df['title'].iloc[sport_indices]

In [113]:
ahoy_events_df.user_id.unique()

array([64, 55, 59, 65])

In [114]:
content_recommender(65)

44                             Canoeing on Ilmenau River
274    Spreewald Canoe Tour: Discover the UNESCO bios...
275    Spreewald Canoe Tour: Discover the UNESCO bios...
617    #Bewegung // Fit in den Frühling - 10 min (Hom...
618    #Bewegung // Fit in den Frühling - 10 min (Hom...
                             ...                        
444                                Bouldering@Petite-Île
445                                Bouldering@Petite-Île
244      Latin Cardio and Dance Outdoor Fitness Training
156                    Bachata Dance Class for Newcomers
173                    Bachata Dance Class for Newcomers
Name: title, Length: 100, dtype: object

In [117]:
#content_recommender("Along the shore of the Elbe to Aumühle (52 km/ hard)")

In [118]:
type(events_df.latitude[0])

numpy.float64

In [119]:
def haversine_vectorized(user_lat,
                         user_lon,
                         event_lat,
                         event_lon):
    """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df
        Computes distance in kms
    """

    lat_1_rad, lon_1_rad = np.radians(user_lat), np.radians(user_lon)
    lat_2_rad, lon_2_rad = np.radians(event_lat), np.radians(event_lon)
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [120]:
haversine_vectorized(48.135124, 11.581981, 47.135124, 12.581981)

134.0816183723369

In [121]:
users_df.id.unique()

array([55., 56., 57., 58., 60., 61., 62., 63., 64., 59., 65., nan])

In [122]:
get_user_loc(65, df = users_df)

NameError: name 'get_user_loc' is not defined

#  recommemder filter for future dates and nearest places 

In [149]:
ahoy_events_df.head(15)

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517
5,17,51,64,View Offer Page ID 1436,"{'user': 64, 'offer': 1436}",2022-05-28 13:39:07.953398
6,18,50,55,View Offer Page ID 1372,"{'user': 55, 'offer': 1372}",2022-05-28 13:54:31.020758
7,19,50,55,View Offer Page ID 1371,"{'user': 55, 'offer': 1371}",2022-05-28 13:54:41.405336
8,20,50,59,View Offer Page ID 1375,"{'user': 59, 'offer': 1375}",2022-05-28 13:55:06.444047
9,21,50,59,View Offer Page ID 1376,"{'user': 59, 'offer': 1376}",2022-05-28 13:55:12.395584


In [124]:
events_df.head()

,id,title,description,price_per_person,capacity,address,latitude,longitude,is_external,url,offer_date,offer_time,created_at,updated_at,user_id,tag_id,external_image_url
0,2336,Gilde Bowling Wandsbek,no description,0,1.0,no address provided,NaN,NaN,True,https://www.meetup.com/bowling-und-freizeit-gr...,2022-06-16,18:45:00,2022-06-03 13:43:47.730864,2022-06-03 13:43:47.730864,NaN,124,https://secure-content.meetupstatic.com/images...
1,2337,Gilde Bowling Wandsbek,no description,0,1.0,no address provided,NaN,NaN,True,https://www.meetup.com/bowling-und-freizeit-gr...,2022-07-07,18:45:00,2022-06-03 13:43:48.211787,2022-06-03 13:43:48.211787,NaN,124,https://secure-content.meetupstatic.com/images...
2,2338,Bowlingabend,Zwei Bahnen sind für uns ab 19 Uhr im Gilde-Bo...,0,NaN,Wandsbeker Zollstraße 25-29,53.575390,10.074991,True,https://www.meetup.com/bowling-gruppe/events/2...,2022-06-25,19:00:00,2022-06-03 13:43:48.706512,2022-06-03 13:43:48.706512,NaN,124,https://secure-content.meetupstatic.com/images...
3,2339,Return To Love Virtual Retreat,"""Jess X Goh guided me in a process that made m...",0,NaN,no address provided,-8.521147,179.196200,True,https://www.meetup.com/the-socializer-hamburg/...,2022-07-22,10:30:00,2022-06-03 13:43:49.208924,2022-06-03 13:43:49.208924,NaN,124,https://secure-content.meetupstatic.com/images...
4,2340,Gaelic Football Training Hamburg m/f/d - all w...,Beginners welcome to join Gaelic football trai...,0,3.0,no address provided,53.594521,10.017610,True,https://www.meetup.com/Irish-Sports-Hurling-Ca...,2022-06-08,18:00:00,2022-06-03 13:43:53.237369,2022-06-03 13:43:53.237369,NaN,125,https://secure-content.meetupstatic.com/images...


In [125]:
from datetime import date

def get_title(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_idx = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    title = events_df[events_df['id']==event_idx].title
    
    return title  

def get_user_loc(user_id, df = users_df):   
    ''' this finction grabs the location of an user from ahoy_events dataset'''
    
    users_idx = users_df[users_df['id']== user_id]
    # now we get the user's latitude and longitude
    
    location_dict = {'user_latitude' : float(users_idx.latitude.values), 
                     
                     'user_longitude' : float(users_idx.longitude.values)
                        }
     
    return location_dict


def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    '''This recommender finds out 20 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    title = get_title(user_id, df = ahoy_events_df)
    
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    df_all_recommendations = df.iloc[sport_indices][[ 'id','title', 'latitude', 'longitude', 'offer_date']]
    
    df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= datetime.date.today()]
    
    #calcullation of haversine distance for users:
    user_latitude = get_user_loc(user_id, df = users_df)['user_latitude']
    user_longitude = get_user_loc(user_id, df = users_df)['user_longitude']
    
    df_date_filter['user_latitude'] = user_latitude
    df_date_filter['user_longitude'] = user_longitude
    
    df_date_filter["distance"] = haversine_vectorized(df_date_filter.latitude, df_date_filter.longitude,
                                                      df_date_filter.user_latitude, df_date_filter.user_longitude)
            
    df_location_filter = df_date_filter[df_date_filter['distance'] <= 200.0]
    
    
                                                
    return df_location_filter.title
#     return user_latitude

In [150]:
ahoy_events_df.user_id.unique()

array([64, 55, 59, 65])

In [127]:
content_recommender(65)

/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/2034847226.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_latitude'] = user_latitude
/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/2034847226.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_longitude'] = user_longitude
/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/2034847226.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

274    Spreewald Canoe Tour: Discover the UNESCO bios...
275    Spreewald Canoe Tour: Discover the UNESCO bios...
273    Spreewald Canoe Tour: Discover the UNESCO bios...
182    Fairytale Forest: Hike through the mystical la...
181    Fairytale Forest: Hike through the mystical la...
183    Fairytale Forest: Hike through the mystical la...
253    Nature Games in Grunewald: Let out your inner ...
179                 Bulgaria  🇧🇬 - Seven Rila lakes hike
178                            Lets hike to Teufelsberg!
99                     Casual Pick-up Women's Basketball
102                    Casual Pick-up Women's Basketball
131    Nature Cycling: Explore one of the most famous...
88                                    Kajak Berlin Tours
194           Join our team - Looking for organizers :) 
788             Seewoog parkun - weekly 5km Run-Walk-Jog
789             Seewoog parkun - weekly 5km Run-Walk-Jog
790             Seewoog parkun - weekly 5km Run-Walk-Jog
147    Four Towers: Top-voted h

# updating event prediction: finalizing stuff

In [151]:
def get_event_id(user_id):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_id = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[-1].properties['offer']
    # now we get the title of the event
    #event_id = events_df[events_df['id']==event_idx].title
    
    return event_id 



In [153]:
ahoy_events_df.user_id.unique()

array([64, 55, 59, 65])

In [154]:
get_event_id(65)

2766

In [158]:
events_df.columns

Index(['id', 'title', 'description', 'price_per_person', 'capacity', 'address',
       'latitude', 'longitude', 'is_external', 'url', 'offer_date',
       'offer_time', 'created_at', 'updated_at', 'user_id', 'tag_id',
       'external_image_url'],
      dtype='object')

In [161]:
events_df[events_df['id']== get_event_id(65)].index



Int64Index([450], dtype='int64')

In [162]:
#id_to_index = pd.Series(events_df.id, index = events_df[get_event_id(65)]).drop_duplicates()

In [131]:
id_to_index

id
2336   NaN
Name: id, dtype: float64

In [138]:
ahoy_events_df.user_id

0     64
1     64
2     64
3     64
4     64
5     64
6     55
7     55
8     59
9     59
10    55
11    55
12    55
13    55
14    55
15    55
16    55
17    55
18    64
19    64
20    64
21    65
22    65
23    65
Name: user_id, dtype: int64

In [ ]:
ahoy_events_df.head()

In [ ]:
users_df.head()

In [166]:
from datetime import date

def get_event_id(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_id = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    #event_id = events_df[events_df['id']==event_idx].title
    
    return event_id 

def get_user_loc(user_id, df = users_df):   
    ''' this finction grabs the location of an user from ahoy_events dataset'''
    
    users_idx = users_df[users_df['id']== user_id]
    # now we get the user's latitude and longitude
    
    location_dict = {'user_latitude' : float(users_idx.latitude.values), 
                     
                     'user_longitude' : float(users_idx.longitude.values)
                        }
     
    return location_dict


def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df):
    '''This recommender finds out 100 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    event_id = get_event_id(user_id, df = ahoy_events_df)
    
    event_idx = events_df[events_df['id']== get_event_id(user_id)].index

    
    sim_scores = list(enumerate(cosine_sim[event_idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    df_all_recommendations = df.iloc[sport_indices][[ 'id','title', 'latitude', 'longitude', 'offer_date']]
    
    df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= datetime.date.today()]
    
    #calcullation of haversine distance for users:
    user_latitude = get_user_loc(user_id, df = users_df)['user_latitude']
    user_longitude = get_user_loc(user_id, df = users_df)['user_longitude']
    
    df_date_filter['user_latitude'] = user_latitude
    df_date_filter['user_longitude'] = user_longitude
    
    df_date_filter["distance"] = haversine_vectorized(df_date_filter.latitude, df_date_filter.longitude,
                                                      df_date_filter.user_latitude, df_date_filter.user_longitude)
            
    df_location_filter = df_date_filter[df_date_filter['distance'] <= 200.0]
    
    
                                                
    return df_location_filter.title
#     return user_latitude

In [167]:
content_recommender(65)

/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/881908717.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_latitude'] = user_latitude
/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/881908717.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_longitude'] = user_longitude
/var/folders/sl/ygshjd2x6x74vgfq6g4z9mk80000gn/T/ipykernel_93488/881908717.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

274    Spreewald Canoe Tour: Discover the UNESCO bios...
275    Spreewald Canoe Tour: Discover the UNESCO bios...
273    Spreewald Canoe Tour: Discover the UNESCO bios...
182    Fairytale Forest: Hike through the mystical la...
181    Fairytale Forest: Hike through the mystical la...
183    Fairytale Forest: Hike through the mystical la...
253    Nature Games in Grunewald: Let out your inner ...
179                 Bulgaria  🇧🇬 - Seven Rila lakes hike
178                            Lets hike to Teufelsberg!
99                     Casual Pick-up Women's Basketball
102                    Casual Pick-up Women's Basketball
131    Nature Cycling: Explore one of the most famous...
88                                    Kajak Berlin Tours
194           Join our team - Looking for organizers :) 
788             Seewoog parkun - weekly 5km Run-Walk-Jog
789             Seewoog parkun - weekly 5km Run-Walk-Jog
790             Seewoog parkun - weekly 5km Run-Walk-Jog
147    Four Towers: Top-voted h